In [1]:
#Importing the required Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
#Loading the dataset
df=pd.read_csv("/content/Advertising.csv")

In [3]:
#Geting the first 5 rows of the data
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
#Getting the overview of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [5]:
#Checking for the nulll values
df.isna().sum()

,0
TV,0
radio,0
newspaper,0
sales,0


In [6]:
#checking for the duplicated vlaued
df.duplicated().sum()

np.int64(0)

In [11]:
#Separating independent and dependent features
x=df.iloc[:,0:3]
y=df.iloc[:,-1]

In [13]:
#Split data into training and testing sets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=3)

In [18]:
#making the objec of the linear model
mlr=LinearRegression()

In [19]:
#Traingn the model
mlr.fit(x_train,y_train)

LinearRegression()

In [24]:
x_test.iloc[0]  # Use .iloc to access the row by its index

,40
TV,202.5
radio,22.3
newspaper,31.6


In [32]:
# Get the feature names from the original DataFrame (df)
feature_names = df.columns[:3]  # Assuming the first 3 columns are features

# Prepare the input data for prediction, including feature names
input_data = pd.DataFrame(x_test.iloc[2].values.reshape(1, -1), columns=feature_names)

# Now make the prediction
prediction = mlr.predict(input_data)
print(prediction)

[19.45830733]


In [33]:
# Regression Metrics
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [34]:
#Predicting based on the test data and saving in a variable
y_pred=mlr.predict(x_test)

In [35]:
print("MAE :- ",mean_absolute_error(y_test,y_pred))

MAE :-  1.1986992501928753


In [36]:
print("MSE :- ",mean_squared_error(y_test,y_pred))

MSE :-  2.153781905661566


In [37]:
print("RMSE- ",np.sqrt(mean_squared_error(y_test,y_pred)))

RMSE-  1.4675768823682003


In [38]:
print("R2 Score :- ",r2_score(y_test,y_pred))

R2 Score :-  0.9138375011080567


In [39]:
adjusted_r2=1-(1-r2_score(y_test,y_pred))*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)

In [40]:
print("Adjusted R2 Score is :- ",adjusted_r2)

Adjusted R2 Score is :-  0.9066572928670614


In [49]:
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

In [47]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [51]:
# Fix Newspaper budget at its mean value
fixed_newspaper = df["newspaper"].mean()

# Create a grid of TV & Radio values
tv_range = np.linspace(df['TV'].min(), df['TV'].max(), 10)
radio_range = np.linspace(df['radio'].min(), df['radio'].max(), 10)
TV_grid, Radio_grid = np.meshgrid(tv_range, radio_range)

# Predict sales using the trained model (mlr)
Newspaper_fixed = np.full(TV_grid.shape, fixed_newspaper)  # Keep Newspaper constant
# Convert to DataFrame with correct column names
features = pd.DataFrame({
    'TV': TV_grid.ravel(),
    'radio': Radio_grid.ravel(),
    'newspaper': Newspaper_fixed.ravel()
})

# Predict using the DataFrame (now it has correct feature names)
Sales_pred = mlr.predict(features)
Sales_pred = Sales_pred.reshape(TV_grid.shape)


# Create 3D interactive plot
fig = go.Figure()

# Add scatter plot for actual data
fig.add_trace(go.Scatter3d(
    x=df['TV'], y=df['radio'], z=df['sales'],
    mode='markers', marker=dict(size=6, color='red', opacity=0.8),
    name="Actual Sales"
))

# Add regression plane
fig.add_trace(go.Surface(
    x=TV_grid, y=Radio_grid, z=Sales_pred,
    colorscale='viridis', opacity=0.6, name="Regression Plane"
))

# Layout settings
fig.update_layout(
    title=f"Interactive 3D Regression Plane (Fixed Newspaper at {fixed_newspaper:.1f})",
    scene=dict(
        xaxis_title="TV Advertising Budget",
        yaxis_title="Radio Advertising Budget",
        zaxis_title="Sales",
    ),
    margin=dict(l=0, r=0, b=0, t=40),
)

# Show interactive plot
fig.show()